**Aluno:** Leonel Mota Sampaio Durão

**Matrícula:** 2019006876

In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
from PIL import Image
from objloader import *

pygame 2.0.1 (SDL 2.0.14, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


Aqui serão importados a matriz de parâmetros intrínsecos e coeficiêntes de distorção da câmera do vídeo que foram previamente calculados, além da leitura do alvo e do vídeo.

In [2]:
intrinsic_matrix =  np.array([[536.41169501,   0.,         317.82103691],
                     [  0.,         536.12110912, 209.10793973],
                     [  0.,           0.,           1.        ]])
dist_coefs = np.array([0.0746, -0.151, -0.0054, 0.00111, 0.0])
INVERSE_MATRIX = np.array([[ 1.0, 1.0, 1.0, 1.0],
                           [-1.0,-1.0,-1.0,-1.0],
                            [-1.0,-1.0,-1.0,-1.0],
                            [ 1.0, 1.0, 1.0, 1.0]])

template = cv2.imread('alvo.jpg',cv2.IMREAD_GRAYSCALE)
ret,template = cv2.threshold(template, 127, 255, cv2.THRESH_BINARY)
h,w = template.shape
extremidades_template =np.array([ [0,0],[w-1,0],[w-1,h-1],[0,h-1] ])
cap = cv2.VideoCapture('entrada.mp4')

### Funções OpenCV

In [3]:
# Recebe os contornos e retorna os que possuem formato retangular com uma área > 1000
def find_squares(contours):
    squares = []
    for cnt in contours: 
        cnt_len = cv2.arcLength(cnt, True)
        cnt = cv2.approxPolyDP(cnt, 0.02*cnt_len, True)
        if len(cnt) == 4 and cv2.contourArea(cnt) > 1000 and cv2.isContourConvex(cnt):
            cnt = cnt.reshape(-1, 2)
            squares.append(np.array(cnt))
    return squares

# recebe duas imagens e retorna a taxa de pixels que não batem
def abs_error(imageA, imageB):
    err = np.sum(np.abs(imageA.astype("float") - imageB.astype("float")))
    err /= float(imageA.shape[0] * imageA.shape[1])*256
    return err


# acharemos a homografia para todos os quadrados encontrados no frame
# Então retificamos e fazemos a similaridade com o template rotacionado de todas as formas.
# para os quadrados retificados que possuam um erro menor que 10% com o alvo, será salvo no
# vetor matches e a sua rotação no vetor pos_matches
def find_matches(squares, img_bin, template, extremidades_template):
    matches = []
    pos_matches = []
    for sq in squares:
        M, mask = cv2.findHomography(sq, extremidades_template, cv2.RANSAC,5.0)
        reproj = cv2.warpPerspective(img_bin,M,(w, h),flags=cv2.INTER_LINEAR)    
        for idx, angle in enumerate([0, 90, 180, 270]):
            M = cv2.getRotationMatrix2D((reproj.shape[1]/2, reproj.shape[0]/2), angle, 1)
            rot = cv2.warpAffine(reproj, M, (reproj.shape[0], reproj.shape[1]))
            error = abs_error(rot, template)
            if error < 0.1:
                matches.append(sq)
                pos_matches.append(idx)
                break
    return matches, pos_matches

# depedendo da rotação do alvo, temos que retornar as extremidades do template em diferentes ordens
def get_pontos(pos):
    if pos == 0:
        return np.array([[1,1,1], [1,-1,1], [-1,-1,1], [-1,1,1]]).astype(float)
    if pos == 1:
        return np.array([[-1,-1,1], [-1,1,1], [1,1,1], [1,-1,1]]).astype(float)
    if pos == 2:
        return np.array([[1,-1,1], [-1,-1,1], [-1,1,1], [1,1,1]]).astype(float)
    if pos == 3:
        return np.array([[-1,1,1], [1,1,1], [1,-1,1], [-1,-1,1]]).astype(float)

# Recebe os alvos detectados em conjunto com os dados intrínsecos da camera para calcular a pose
def find_poses(matches, pos_matches, intrinsic_matrix, dist_coefs):
    result = []
    for i, m in enumerate(matches):
        ret,rvecs, tvecs = cv2.solvePnP(get_pontos(pos_matches[i]), m.astype(float), intrinsic_matrix, dist_coefs)
        if ret:
            result.append({'ret':ret,'rvecs':rvecs,'tvecs':tvecs})
    return result

# Essa função funciona como a função principal para a parte de OpenCV
# ela recebe um frame e faz todo o processo de binarização, detecção de bordas, detecção de contornos
# então a função que encontra quadrados a partir de contornos é chamada
# e o resultado é dado para a função que retifica e faz o template matching
# em seguida, são calculadas as poses dos alvos encontrados
def draw_matches_and_find_poses(img):
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret,img_bin = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
    bordas = cv2.Canny(img,100,200)
    contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    squares = find_squares(contours)
    matches, pos_matches = find_matches(squares, img_bin, template, extremidades_template)
    poses = find_poses  (matches, pos_matches, intrinsic_matrix, dist_coefs)
    cv2.drawContours(img, matches, -1, (0,255,0), 3)
    return poses

### OpenGL


In [4]:
def initOpenGL(dimensions):

    (width, height) = dimensions
    
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glClearDepth(1.0)
    glEnable(GL_TEXTURE_2D)
    glEnable(GL_DEPTH_TEST)
    obj = OBJ('Pikachu.obj', swapyz=True)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
 
    fovy = 45
    aspect = (width)/(height)
    gluPerspective(fovy, aspect, 0.1, 100.0)
    return obj
 
# renderiza cubo e pikachu
def show_obj(view_matrix):
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()            
    glEnable(GL_TEXTURE_2D)

    glPushMatrix()
    glLoadMatrixf(view_matrix)
    #glScalef(0.5, 0.5, 0.5);
    glTranslatef(0.0, 0.0, 2.50)

    glPushMatrix()
    glTranslatef(0.0, 0.0, 0.0)
    glutWireCube(2.0)
    glPopMatrix()

    glCallList(obj.gl_list)

    glPopMatrix()

# Renderiza imagem no fundo do objeto 
def show_img(img):

    textureId = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, textureId)
    background = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    background = cv2.flip(background, 0)
    height, width, channels = background.shape
    background = np.fromstring(background.tostring(), dtype=background.dtype, count = height * width * channels)    
    background.shape = (height, width, channels)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, background)
    glDepthMask(GL_FALSE)
    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, width, 0, height)
    glEnable(GL_TEXTURE_2D)
    glBindTexture(GL_TEXTURE_2D, textureId)
    glMatrixMode(GL_MODELVIEW)
    glPushMatrix()
    glBegin(GL_QUADS)
    glTexCoord2f(0, 0); glVertex2f(0, 0)
    glTexCoord2f(1, 0); glVertex2f(width, 0)
    glTexCoord2f(1, 1); glVertex2f(width, height)
    glTexCoord2f(0, 1); glVertex2f(0, height)
    glEnd()
    glPopMatrix()
    glMatrixMode(GL_PROJECTION)
    glPopMatrix()
    glMatrixMode(GL_MODELVIEW)
    glBindTexture(GL_TEXTURE_2D, 0)
    glDepthMask(GL_TRUE)
    glFlush()
    

# recebe o resultado da função solvePnP e cria a função de rotação a partir do rvec e concatena o tvec
# em seguida, multiplica a segunda e terceira linha por -1, transpõe e retorna
def calculate_view_matrix(pose):
    rvec = pose['rvecs']; tvec = pose['tvecs']
    x = tvec[0][0]; y = tvec[1][0]; z = tvec[2][0];

    rmtx = cv2.Rodrigues(rvec)[0]
    view_matrix = np.array([[rmtx[0][0], rmtx[0][1], rmtx[0][2], x],        
                            [rmtx[1][0], rmtx[1][1], rmtx[1][2], y],
                            [rmtx[2][0], rmtx[2][1], rmtx[2][2], z], 
                            [0.0       ,0.0       ,0.0       ,1.0]])


    view_matrix = view_matrix * INVERSE_MATRIX                                           
    view_matrix = np.transpose(view_matrix)
    return view_matrix

# para cada frame, chama a função principal da parte de OpenCV que retorna a pose dos alvos detectados
# en seguida, chama a função que renderiza o frame como fundo e então renderiza os objetos
def displayCallback():
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    flag,img = cap.read()
    if flag:
        show_img(img)
        poses = draw_matches_and_find_poses(img)
        for pose in poses:
            view_matrix = calculate_view_matrix(pose)
            show_obj(view_matrix)
    glutSwapBuffers()    
    

def idleCallback():
    glutPostRedisplay()
    
    

dimensions = (640, 480)
glutInit()
glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE)
glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION)
glutInitWindowSize(*dimensions)
window = glutCreateWindow(b'Realidade Aumentada [codigo esqueleto]')

obj = initOpenGL(dimensions)

glutDisplayFunc(displayCallback)
glutIdleFunc(idleCallback)

glutMainLoop()

<ipython-input-4-87bafa77c16f>:46: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  background = np.fromstring(background.tostring(), dtype=background.dtype, count = height * width * channels)
<ipython-input-4-87bafa77c16f>:46: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  background = np.fromstring(background.tostring(), dtype=background.dtype, count = height * width * channels)
